In [1]:
!pip install sentence_transformers
!pip install ftfy
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/54.4 kB ? eta -:--:--
   ------- -------------------------------- 10.2/54.4 kB ? eta -:--:--
   ---------------------- ----------------- 30.7/54.4 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 54.4/54.4 kB 566.6 kB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/235.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/235.5 kB ? eta -:--:--
   ----------------------------- ---------- 174.1/235.5 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 235.5/235.5 kB 3.6 MB/s eta 0:00:00


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import re
from sentence_transformers import util
import torch
import string
import chromadb

c:\Users\Lenovo\anaconda3\envs\ChromaDB2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('C:\TFG\DataProcessed\independencia_df_es.csv')

In [3]:
def preprocess_text_independence_es(tweet):
    '''
    la función preprocess_text_independence recibe un tweet y realiza una serie de transformaciones
    parámetros:
    - tweet: texto del tweet en español
    retorna:
    - text: texto preprocesado en español
    '''
    text = re.sub(r'@\w+|#\w+', '', tweet)
    # se eliminan los signos de puntuación de cada tweet
    text = "".join([char for char in text if char not in string.punctuation])
    emojis = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticonos
                        u"\U0001F300-\U0001F5FF"  # símbolos y emoticonos
                        u"\U0001F680-\U0001F6FF"  # emoticonos suplementarios
                        u"\U0001F900-\U0001F9FF"  # emoticonos ideográficos
                        u"\U0001FA00-\U0001FA6F"  # emoticonos de símbolos diversos
                        u"\U0001FA70-\U0001FAFF"  # emoticonos de transporte y objetos
                        u"\U0001F000-\U0001F0FF"  
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    text = emojis.sub('', text)
    # se eliminan las URL que empiezan por http
    text = re.sub(r'http\S+', '', text)
    # se eliminan las URL que empiezan por www
    text = re.sub(r'www\S+', '', text)
    # se eliminan las URL que empiezan por https
    text = re.sub(r'https\S+', '', text)
    return text

In [4]:
# se aplica la función preprocess_text_independence al dataset en español
df['Texto_preprocesado'] = df['Texto'].apply(lambda x: preprocess_text_independence_es(x))

In [5]:
df

,Id,Texto,Sentimiento,Texto_preprocesado
0,1.099284e+18,RT @EFEnoticias: Arrimadas se presenta a las g...,Negativo,RT Arrimadas se presenta a las generales sa...
1,1.102570e+18,@gabrielrufian Derecho a la autodeterminación ...,Negativo,Derecho a la autodeterminación Golpe de esta...
2,1.097577e+18,#URGENTE ???? #Cataluña: Los CDR atacan con pi...,Negativo,Los CDR atacan con pintura el cuartel de la...
3,1.097476e+18,Cansado de los eslóganes de izquierdas y derec...,Negativo,Cansado de los eslóganes de izquierdas y derec...
4,1.093134e+18,Le recuerdo a @EnricJuliana que en Cataluña lo...,Neutral,Le recuerdo a que en Cataluña los no indepes ...
...,...,...,...,...
10072,1.102501e+18,La internacionalización del procés implica u...,Negativo,La internacionalización del procés implica u...
10073,1.098853e+18,Hace 5 días salieron a la calle 500.000 catala...,Positivo,Hace 5 días salieron a la calle 500000 catalan...
10074,1.099084e+18,RT @miqueldelpozo: Nuestro PIB (Producto Inte...,Neutral,RT Nuestro PIB Producto Interior Bruto no mi...
10075,1.097772e+18,? #ÚltimaHora Comienza la cuarta jornada del #...,Negativo,Comienza la cuarta jornada del con más inte...


In [6]:
df = df[['Texto_preprocesado', 'Sentimiento']]

In [7]:
# se guarda el data set
df.to_csv('C:\TFG\DataProcessed\independencia_df_es_retrieval.csv', index=False)

### Versión CHROMADB + Langchain + Fine-Tuning

In [8]:
client = chromadb.PersistentClient(path="./database")

In [10]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="hiiamsid/sentence_similarity_spanish_es")
collection = client.get_or_create_collection(name="tweets")

c:\Users\Lenovo\anaconda3\envs\ChromaDB2\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
documents = []
metadatas = []
ids = []
for j, item in enumerate(df[['Sentimiento', 'Texto_preprocesado']].values):
    ids.append(str(j))
    documents.append(item[1])
    metadatas.append({"title": item[0]})

collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
In

In [10]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="hiiamsid/sentence_similarity_spanish_es")

In [11]:
from langchain.vectorstores import Chroma

langchain_chroma = Chroma(
    client=client,
    collection_name="tweets",
    embedding_function=embedding_function,
)
print("There are", langchain_chroma._collection.count(), "contexts in the collection")

There are 10077 contexts in the collection


In [12]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

Is CUDA available: True
CUDA version: 12.1
cuDNN version: 8801


In [14]:
results = collection.query(
    query_texts=["¿que opinan sobre la independencia de cataluña?"],
    n_results=5,
    include=["documents"]
)
results['documents']

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embe

[[' Cuánto odio ¿no ¿entonces la mitad de Cataluña odia a la mitad de Cataluña pues van pidiendo neutralidad y quitando lazos amarillos solidarios con aquellos que intentaron romper la democracia',
  '  Solo ganarán en Cataluña porque los indepes nos quedaremos en casa',
  ' Dastis Borrell y Lozano están haciendo tanto por la independencia de Catalunya como nosotros o más',
  ' Ud es extranjero y apoya la independencia de Cataluña me puede explicar cuáles son los motivos que le llevan a posicionarse a favor de esa causa',
  'Lo de Cataluña no tiene remedio Los independentistas no van a parar hasta conseguir lo que quieren Y si como espero nunca lo consiguen nunca dejarán de intentarlo']]

In [33]:
title = "Positivo"

results = collection.query(
    query_texts=["Qué piensan los twitteros sobre Puigdemont?"],
    n_results=10,
    where={"title": title}
)
results

{'ids': [['9671',
   '6675',
   '999',
   '3535',
   '8123',
   '7962',
   '2337',
   '1772',
   '9056',
   '8952']],
 'distances': [[0.6439956426620483,
   0.6527489423751831,
   0.668475329875946,
   0.67536461353302,
   0.6778125762939453,
   0.6941722631454468,
   0.6982995271682739,
   0.7005260586738586,
   0.7019259929656982,
   0.7079191207885742]],
 'metadatas': [[{'title': 'Positivo'},
   {'title': 'Positivo'},
   {'title': 'Positivo'},
   {'title': 'Positivo'},
   {'title': 'Positivo'},
   {'title': 'Positivo'},
   {'title': 'Positivo'},
   {'title': 'Positivo'},
   {'title': 'Positivo'},
   {'title': 'Positivo'}]],
 'embeddings': None,
 'documents': [[' Vota y participa en la encuesta del día   ¿Crees que Arrimadas tendría que entrevistarse con Puigdemont  ',
   ' A los exiliados sí La broma fea de Puigdemont y la mejor película extranjera',
   'RT   El impresentable que ha insultado a MHP PUIGDEMONT es este  ',
   'La NVA ve  sorprendente  que PSOE PP y Cs quieran prohibir

In [35]:
collection.query(
    query_texts=["¿Qué mensajes de odio podemos encontrar?"],
    n_results=3,
    where_document= {"$contains": "independencia"}
)

{'ids': [['5256', '351', '1440']],
 'distances': [[0.7375088334083557, 0.7568939328193665, 0.7714074850082397]],
 'metadatas': [[{'title': 'Positivo'},
   {'title': 'Positivo'},
   {'title': 'Positivo'}]],
 'embeddings': None,
 'documents': [['Esta pregunta es un poco tramposa jajaja y te diré porque con la independencia tengo las dos  Jijijijiji lo habías pensado ',
   '  Menos mal que hay más partidos para votar a la mierda ERC y PdCat Actualmente están cagados e inservibles para la independencia Los de la crida muy valientes pero con el mando a distancia ',
   '                                                 Yo veo mas factible dejar la independencia en manos de Lourdes que en mano de estos   Oye despues de  años sin conseguir nada por probar ']],
 'uris': None,
 'data': None}

### Version SBert + Fine-Tuning

In [15]:
# Cargar el modelo,  el parametro device='cuda' sirve para usar la GPU, pero no tenemos acceso
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es', device='cuda')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

In [16]:
# Función de búsqueda semántica
def semantic_search(query, df, column='Texto_preprocesado', top_n=5):
    '''
    la funcion semantic_search toma una consulta y un dataframe y devuelve los resultados más similares en el dataframe
    parámetros:
    - query: texto de la consulta
    - df: dataframe con los textos a comparar
    - column: nombre de la columna que contiene los textos
    - top_n: número de resultados a devolver
    retorna:
    - dataframe con los resultados más similares
    '''
    # Codifica el texto de la consulta
    query_embedding = model.encode([query], convert_to_tensor=True).cpu()
    # Codifica los textos del dataframe
    text_embeddings = model.encode(df[column].tolist(), convert_to_tensor=True).cpu()
    # Calcula las puntuaciones de similitud
    cos_scores = util.pytorch_cos_sim(query_embedding, text_embeddings)[0]
    # Obtiene los índices de los resultados más altos
    top_results = torch.topk(cos_scores, k=top_n)
    return df.iloc[top_results.indices].assign(similarity=top_results.values)

In [17]:
# Codifica el texto de la consulta
query_text = "¿que opinan sobre la independencia de cataluña?"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.6082
  A ver independizar toda catalunya menos un trocito que se quedaría en España para que vivieran los de erc y pudieran seguir pidiendo la independencia de ese trocito por los siglos de los siglos   Por mi vale
Similarity Score: 0.6057
 Ud es extranjero y apoya la independencia de Cataluña me puede explicar cuáles son los motivos que le llevan a posicionarse a favor de esa causa
Similarity Score: 0.6000
  Yo soy castellano nacido en la cuidad más española q puedas imaginar hijo de Martín y Lope y además estoy a favor d q los Catalanes decidan su futuro y a favor d la independencia d Cataluña No veo qué problema ves en ser castellano e independentista ¿Me lo aclaras por favor
Similarity Score: 0.5895
Turull Entre un 7080 de la sociedad Catalana quiere votar el futuro de Catalunya y dentro de este  existen personas que defienden un referéndum pero están en contra de la independencia en la Europa del s21 la gente entiende que debe votar para escoger el futuro de su

In [76]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
9319,ud es extranjero y apoya la independencia de ...,Negativo
924,catalan es quien tiene la ciudadania catalana ...,Negativo
3083,yo soy castellano nacido en la cuidad mas es...,Positivo
6243,me considero preso politico erc tiene anos de...,Positivo
5588,a ver independizar toda catalunya menos un t...,Positivo


In [77]:
# Codifica el texto de la consulta
query_text = "¿que opinan sobre el separatismo?"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.5595
     y en erc hay nacionalistas que nunca votarian irse de espana y separatistas y eso que es muy simple solo los totalitarios van en bloque y sois  suerte
Similarity Score: 0.5593
somos muchos antifascistas de izquierdas e independentistas que por mucho que queramos la independencia no vamos a votar a partidos como el pdcat o la crida a si pues senores politicos aparten sus sillones y sumen sus fuerzas     
Similarity Score: 0.5581
    porque en todo lo que sea ir contra catalunya y el independentismo sigue vigente la unidad de accion con sus antiguos socios del  aunque ello represente una alianza con la extrema derecha fascistoide de vox
Similarity Score: 0.5569
rt    mi opinion sobre no votar esa actitud es la derrota del independentismo y la democracia  
Similarity Score: 0.5565
  representa la unica opcion de la izquierda no independentista de    


In [78]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
7320,y en erc hay nacionalistas que nunca vota...,Negativo
2820,somos muchos antifascistas de izquierdas e ind...,Positivo
6561,porque en todo lo que sea ir contra catalu...,Positivo
7315,rt mi opinion sobre no votar esa actitud es...,Positivo
6532,representa la unica opcion de la izquierda n...,Negativo


In [79]:
# Codifica el texto de la consulta
query_text = "¿qué tweets contienen mensajes con sentimientos neutrales?"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.4119
rt  puigdemont ofrece a arrimadas "una entrevista cordial" en waterloo y ciudadanos la rechaza  via ...
Similarity Score: 0.3970
em  a turma politicamente correta atacou donald trump por simplesmente ter denunciado atrocidades de radicais islamicos mostrando videos no twitter  agora guardadas as proporcoes faz o mesmo contra bolsonaro  a esquerda adora culpar o mensageiro nunca o autor do crime
Similarity Score: 0.3931
 el supremo permite a los procesados por el o declarar en catalan  por razones emocionales   informa 
Similarity Score: 0.3923
rt   un buen sitio para llenarlo de sentimientos y democracia espanola aunque se diga lo contrario  vamoooss  saludos...
Similarity Score: 0.3865
me recuerda mucho al argumentario de puigdemont y los indepes "queremos dialogo pero la supremacia de los eeuu" esa forma de mentir dialogo de sordos 


In [80]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
2832,"rt puigdemont ofrece a arrimadas ""una entrevi...",Negativo
3864,em a turma politicamente correta atacou donal...,Neutral
1447,el supremo permite a los procesados por el o ...,Positivo
6482,rt un buen sitio para llenarlo de sentimient...,Negativo
9902,me recuerda mucho al argumentario de puigdemon...,Negativo


In [81]:
# Codifica el texto de la consulta
query_text = "¿que mensajes contienen odio hacia las instituciones catalanas?"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.5643
 cuanto odio ?no ?entonces la mitad de cataluna odia a la mitad de cataluna pues van pidiendo neutralidad y quitando lazos amarillos solidarios con aquellos que intentaron romper la democracia
Similarity Score: 0.5383
condeno una vez mas estos actos intolerables el nacionalismo provoca violencia y odio en cataluna tenemos que acabar con eso con la fuerza de la democracia y del estado de derecho mi solidaridad con el ciudadano agredido y con  en girona  
Similarity Score: 0.5086
                                                  es muy inquietante encontrar en la  maldita hemeroteca  un gran numero de documentos procedentes del nacionalismo catalan con expresiones y simbologia fascista y racista 
Similarity Score: 0.4931
    los matones que se creen con el poder de decir quien puede pisar y quien no las universidades catalanas merecen una censura social en twitter ya esta bien de fascismo separatista   expresemosles nuestra disconformidad a estos totalitarios de 

In [82]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
2463,cuanto odio ?no ?entonces la mitad de catalun...,Negativo
5266,condeno una vez mas estos actos intolerables e...,Negativo
6182,...,Negativo
5247,los matones que se creen con el poder de d...,Negativo
8085,lo preocupante es la gente que en virtud de...,Negativo


In [83]:
# Codifica el texto de la consulta
query_text = "¿qué tweets contienen mensajes con sentimientos negativos?"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.3986
rt  muchas relaciones amorosas estan basadas en un ideario que las conduce al sufrimiento 
Similarity Score: 0.3901
rt  la gente que bloquea a otros cuando se enfadan en vez de hablar las cosas
Similarity Score: 0.3815
usuarios independentistas y de izquierdas denuncian que twitter espana les quita seguidores y retweets  via 
Similarity Score: 0.3645
muchas relaciones amorosas estan basadas en un ideario que las conduce al sufrimiento 
Similarity Score: 0.3596
?como la que recibe ines arrimadas del cibertrol toni alba ?o va a decirnos que eso es humor  por que no he visto su critica al citado 


In [84]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
2248,rt muchas relaciones amorosas estan basadas e...,Neutral
239,rt la gente que bloquea a otros cuando se enf...,Neutral
5131,usuarios independentistas y de izquierdas denu...,Positivo
7228,muchas relaciones amorosas estan basadas en un...,Neutral
264,?como la que recibe ines arrimadas del cibertr...,Negativo


In [85]:
# Codifica el texto de la consulta
query_text = "¿qué tweets contienen mensajes con sentimientos positivos?"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.4235
 eu amo demais serio me sinto mt feliz
Similarity Score: 0.3866
rt   controlar meus sentimentos de verdade porque so assim eu nao iria me magoar tanto 
Similarity Score: 0.3857
rt  amigos   esse e tweet da dor   eu nunca chorei tanto em um episodio 
Similarity Score: 0.3853
el hombre del sofa se muestra comprensivo con la huelga indepe  con algo se han de entretener los cdr jubilados ?no   
Similarity Score: 0.3845
rt   un buen sitio para llenarlo de sentimientos y democracia espanola aunque se diga lo contrario  vamoooss  saludos...


In [86]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
7601,eu amo demais serio me sinto mt feliz,Neutral
1433,rt controlar meus sentimentos de verdade por...,Neutral
9890,rt amigos esse e tweet da dor eu nunca ch...,Neutral
7890,el hombre del sofa se muestra comprensivo con ...,Negativo
6482,rt un buen sitio para llenarlo de sentimient...,Negativo
